## 交叉验证

In [1]:
import numpy as np
from sklearn import datasets

In [2]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

## 测试 train_test_split

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666)

In [6]:
from sklearn.neighbors import KNeighborsClassifier

best_score, best_p, best_k = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights="distance", p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_score = score
            best_p = p
            best_k = k
print("best_k = ", best_k)
print("best_p = ", best_p)
print("best_score = ", best_score)

best_k =  3
best_p =  4
best_score =  0.9860917941585535


## 使用交叉验证进行超参数调整

In [7]:
from sklearn.model_selection import cross_val_score

knn_clf = KNeighborsClassifier()
cross_val_score(knn_clf, X_train, y_train)

array([0.98895028, 0.97777778, 0.96629213])

我们的结果得到一个数组，里面有三个score，说明cross默认将我们的训练数据集分成3份进行验证。

In [8]:
from sklearn.neighbors import KNeighborsClassifier

best_score, best_p, best_k = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights="distance", p=p)
        scores = cross_val_score(knn_clf, X_train, y_train) #得到一个数组
        score = np.mean(scores)
        if score > best_score:
            best_score = score
            best_p = p
            best_k = k
print("best_k = ", best_k)
print("best_p = ", best_p)
print("best_score = ", best_score)

best_k =  2
best_p =  2
best_score =  0.9823599874006478


这里得到的 best_score 就是我们最好的准确率吗？当然不是，我们进行交叉验证的目的只是为了拿到最好的 k 和 p 而已。

In [9]:
best_knn_clf = KNeighborsClassifier(n_neighbors=2, weights="distance", p=2)
best_knn_clf.fit(X_train, y_train)
best_knn_clf.score(X_test, y_test)

0.980528511821975

此时得到的0.98才能说是我们模型的分类准确度

## 回顾网格搜索

In [11]:
from sklearn.grid_search import GridSearchCV # GridSearchCV：这个CV就是交叉验证
param_gird = [
    {
        "weights" : ["distance"],
        "n_neighbors": [i for i in range(2, 11)],
        "p" : [i for i in range(1, 6)]
    }
]
grid_search = GridSearchCV(knn_clf, param_gird, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:   25.4s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=5,
           weights='distance'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'weights': ['distance'], 'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [12]:
grid_search.best_params_

{'n_neighbors': 2, 'p': 2, 'weights': 'distance'}

In [13]:
grid_search.best_score_

0.9823747680890538

In [14]:
best_knn_clf = grid_search.best_estimator_
best_knn_clf.score(X_test, y_test)

0.980528511821975

In [15]:
#交叉验证默认将训练数据集分成三份，如果我们想分成5份，里面有一个cv的参数
cross_val_score(knn_clf, X_train, y_train, cv=5)

array([0.99543379, 0.96803653, 0.98148148, 0.96261682, 0.97619048])